In [ ]:
!pip install implicit

     |████████████████████████████████| 1.1MB 13.7MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419445 sha256=24c31cf4efae98d11d1e6f3f9afb003e97e6b01121f43e5bf058732f06c12816
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
!unzip retail_train.zip

Archive:  retail_train.zip
  inflating: retail_train.csv        


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
**На вебинаре мы использовали товар 999999 - что это за товар?**

*Искуственно созданный товар*

**Зачем он нужен?**

*Чтобы не потерять пользователей в матрице, которые не покупали товары из топ-5000*

**Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?**

*В сторону этого товара, он будет присутствовать для большинства пользователей, и следователько попадать почти во все рекомендации*

Можно ли удалить этот товар?

Уберите этот товар и сравните с качеством на семинаре.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [ ]:
data_train.item_id.nunique()

86865

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n,replace=False, p = weights)
    
    
    return recs.tolist()

In [ ]:
#По кол-ву проданных штук
items_weights_quantity = data_train.groupby('item_id')['quantity'].sum().reset_index()
items_weights_quantity['weight'] = items_weights_quantity['quantity'].apply(lambda x: np.log(x) if x != 0 else x)
items_weights_quantity['weight'] = items_weights_quantity['weight'] / items_weights_quantity['weight'].sum()

#По сумме продаж
items_weights_sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights_sales['weight'] = items_weights_sales['sales_value'].apply(lambda x: abs(np.log(x)) if x != 0 else x)
items_weights_sales['weight'] = items_weights_sales['weight'] / items_weights_sales['weight'].sum()

#По кол-ву отдельных продаж (добавил единицу, чтобы не было 0, если проданна одна штука)
items_weights_counts = data_train.groupby('item_id')['sales_value'].count().reset_index()
items_weights_counts['weight'] = items_weights_counts['sales_value'].apply(lambda x: abs(np.log(x + 1)) if x != 0 else x)
items_weights_counts['weight'] = items_weights_counts['weight'] / items_weights_counts['weight'].sum()
items_weights_counts

,item_id,sales_value,weight
0,25671,3,0.000009
1,26081,1,0.000004
2,26093,1,0.000004
3,26190,1,0.000004
4,26355,1,0.000004
...,...,...,...
86860,17381856,1,0.000004
86861,17382205,1,0.000004
86862,17383227,1,0.000004
86863,17827644,1,0.000004


Сделайте предсказания

In [ ]:
%%time

# your_code
result['weighted_random_recommendation_quantity'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_quantity, n=5))
result['weighted_random_recommendation_sales'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_sales, n=5))
result['weighted_random_recommendation_counts'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_counts, n=5))

CPU times: user 9.64 s, sys: 204 ms, total: 9.84 s
Wall time: 9.86 s


In [ ]:
result.head(2)

,user_id,actual,weighted_random_recommendation_quantity,weighted_random_recommendation_sales,weighted_random_recommendation_counts
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9527615, 983936, 951299, 787798, 10457483]","[12188573, 1523514, 938292, 15687116, 889355]","[9297504, 1050441, 923403, 1013259, 9487003]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1126252, 945385, 6469294, 13072796, 12810242]","[9484494, 5569301, 1002787, 837509, 1130323]","[1086644, 826830, 10250438, 9858866, 1111032]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [ ]:
result_webinar = pd.read_csv('preds.csv')# закгрузка predict с семианара
result_webinar.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8067309, 13189944, 874071, 1130944, 1885366]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12263501, 982982, 395388, 1101418, 15927781]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[903398, 801647, 859730, 1012444, 6975728]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 961554]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[931285, 12811006, 136217, 1111665, 10456979]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 826249]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13072680, 913958, 982985, 13416321, 1062425]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 1098066, 995242]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 1098066, 826249, 1127831]"


In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [ ]:
# Метрика по семинару
precision_all = []
result_webinar['actual'] = result_webinar['actual'].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))
for column in result_webinar.columns[2:]:
  result_webinar[column] = result_webinar[column].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))
  precision_all.append(round(result_webinar.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean(), 4))


In [ ]:
for column in result.columns[2:]:
  precision_all.append(round(result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean(), 4))

In [ ]:
columns = result_webinar.columns[2:].to_list() + result.columns[2:].to_list()
pd.Series(data = precision_all, index = columns)

random_recommendation                      0.0004
popular_recommendation                     0.1552
itemitem                                   0.1512
cosine                                     0.1551
tfidf                                      0.1609
weighted_random_recommendation_quantity    0.0030
weighted_random_recommendation_sales       0.0017
weighted_random_recommendation_counts      0.0015
dtype: float64

Лучший результат показывает tfidf и другие детерминированные алгоритмы, но что интересно popular_recom тоже не плохо себя показывает


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


Random Recommendation Top-5000

In [ ]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [ ]:
%%time
top5000 = data_train.groupby('item_id')['quantity'].sum().reset_index()
top5000.sort_values('quantity', ascending=False, inplace = True)
items = top5000.head(5000).item_id.to_list()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 1.06 s


In [ ]:
precision_all.append(round(result.apply(lambda x: precision_at_k(x['random_recommendation_5000'], x['actual'],  5), axis=1).mean(), 4))

In [ ]:
columns = result_webinar.columns[2:].to_list() + result.columns[2:].to_list()
all_metrics = pd.Series(data = precision_all, index = columns)

In [ ]:
all_metrics[['random_recommendation', 'random_recommendation_5000']]

random_recommendation         0.0004
random_recommendation_5000    0.0071
dtype: float64

Baseline используя Top-5000 для рандом рекомендации улучшился

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.